'''
이 파일은 read.me 같은 설명서 
모델의 약점은 제일 마지막에 설명되어잇음 
'''

'''
내가 사용한 전처리 이미지 전처리 부분 

단, 예를 들어 사용한다면 꼭 train = False를 매개변수로 써서 활용할것! 

val_transforms = CustomTransforms(train=False)

'''

In [ ]:
class CustomTransforms:
    def __init__(self, train=True):
        if train:
            self.transforms = transforms.Compose([
                transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transforms = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def __call__(self, img):
        return self.transforms(img)

In [ ]:
val_transforms = CustomTransforms(train=False)

'''
저장된 모델을 불러올때는 쓰는 부분은 밑에 거 처럼 하면 됌
결국 제일 마지막에 한 리니어 층 한개로 한개 일단 제일 성능이 잘 나왓음. 
약 0.7 정도임 
'''

In [ ]:
# 저장된 모델 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.vgg16(pretrained=False)
model.classifier[6] = torch.nn.Linear(4096, 1)
model.load_state_dict(torch.load('vgg16_cheetah_classifier.pth', map_location=device))
model = model.to(device)
model.eval()

'''
그리고 예측 값을 보기 위해서  sigmoid를 써서 0.5 이하라면 0 / 0.5 이상이라면 1이나오게끔 하는 함수 
'''

In [ ]:
def predict_image(image):
    image = val_transforms(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()
    return 1 if prediction >= 0.5 else 0

'''
일단 문양 구분이 부분이 매우 부족함
따라서 아기 표범의 경우 구분을 매우 못함

두번째 늘씬한 형태로 점박이가 되어 있으면 치타라 인식할 가능성이 잇음

세번째 위의 것과 연관되는 것으로 늘씬한 형태가 아니면 예를 들어 치타의 얼굴이 있다면 치타가 아니라고 할 가능성이 높음
'''